In [7]:
# ====================================================
# <---- COLD EMAIL HUNTER — Level 2 (CSV Export) ---->
# ====================================================

import os
import csv
import pdfplumber
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import BaseTool
from crewai_tools import SerperDevTool, FileReadTool

In [ ]:
# ====================================================
# 1. CONFIGURATION
# ====================================================

#  API KEYS 
os.environ["GOOGLE_API_KEY"] = "" # Your Google Gemini API Key
os.environ["SERPER_API_KEY"] = "" # Your Serper.dev API Key


agent_llm = LLM(
    model="gemini/gemini-2.5-flash",
    api_key=os.environ["GOOGLE_API_KEY"],
    temperature=0.1
)


In [ ]:
# ====================================================
# 2. FILE SETUP
# ====================================================
pdf_path = "resume.pdf" # Path to your resume PDF
txt_path = "resume.txt"
csv_path = "leads.csv"

# Initialize CSV with headers
if not os.path.exists(csv_path):
    with open(csv_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Company", "Email", "Role/Context", "Source"])

def prepare_resume():
    if not os.path.exists(txt_path):
        if os.path.exists(pdf_path):
            print(" Converting PDF to Text...")
            with pdfplumber.open(pdf_path) as pdf:
                text = "\n".join([p.extract_text() or "" for p in pdf.pages])
            with open(txt_path, "w", encoding="utf-8") as f:
                f.write(text)
    print(f"✅ Resume Context: {txt_path}")

prepare_resume()

✅ Resume Context: resume.txt


In [10]:
# ====================================================
# 3. TOOLS
# ====================================================

class CSVExportTool(BaseTool):
    name: str = "Lead Saver"
    description: str = "Saves a verified lead to leads.csv. Input format: 'Company|Email|Context|Source'"

    def _run(self, lead_string: str) -> str:
        try:
            # Parse the pipe-separated string
            parts = lead_string.split('|')
            if len(parts) < 2:
                return "Error: Input must be 'Company|Email|Context|Source'"

            # Clean whitespace
            cleaned_parts = [p.strip() for p in parts]

            # Append to CSV
            with open(csv_path, mode='a', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                writer.writerow(cleaned_parts)

            return f"✅ Saved lead: {cleaned_parts[1]}"
        except Exception as e:
            return f"Error saving CSV: {str(e)}"

# Initialize Tools
search_tool = SerperDevTool(n_results=15)
file_tool = FileReadTool(file_path=txt_path)
csv_tool = CSVExportTool()


In [11]:
# ====================================================
# 4. AGENTS  (FULL UPGRADED VERSION)
# ====================================================

# 1) ROLE DISCOVERY AGENT
role_discovery = Agent(
    role="Role Discovery Analyst",
    goal="Read resume.txt and infer the top 3–5 job roles best suited for the candidate.",
    
    backstory="""
    You analyze resumes like an ATS + senior recruiter.
    You identify:
      • Core technical skills
      • Project domains
      • Work experience
      • Tools and technologies

    You infer job roles and produce:
      1. Ranked list of 3–5 job roles
      2. Per-role keyword clusters
      3. Search-ready Boolean query strings
    """,
    
    tools=[file_tool],
    llm=agent_llm,
    verbose=True
)


# 2) MULTI-PLATFORM LEAD HUNTER
lead_hunter = Agent(
    role="Recruiter Email Extraction Specialist",

    goal="""
    Find VERIFIED recruiter/hiring manager emails from MULTI-PLATFORM searches.
    Only return emails from:
        • Recruiters
        • HR Managers
        • Talent Acquisition (TA)
        • Founders / Hiring Team
        • Company career posts
    NEVER return emails from:
        • Commenters
        • Students
        • Job seekers
        • 'Interested sir' comments
        • Unrelated Gmail addresses
    """,

    backstory="""
    You ONLY trust emails appearing in SERP snippets where the surrounding
    text clearly shows that the person posting is a RECRUITER or an AUTHORIZED
    hiring representative.

    STRICT FILTERING:
    Accept ONLY if the snippet contains ANY of these recruiter indicators:
        ["HR", "HR Manager", "Recruiter", "Talent Acquisition",
         "Hiring Manager", "TA Team", "Careers@", "Jobs@", "Hiring@", 
         "Founder", "Team", "Company"]
    
    REJECT the snippet if it contains ANY of the following spam indicators:
        ["interested", "please hire me", "my resume", "I want this job",
         "looking for job", "freshers", "help me", "sir please", "I need work"]
    
    You search using 20–30 advanced Google queries per role across:
        • LinkedIn posts (company or recruiter posts only)
        • LinkedIn recruiter profiles
        • Naukri job posts
        • Indeed job descriptions
        • Foundit recruiter posts
        • Glassdoor job listings
        • Twitter recruiter posts
        • Company hiring announcements

    You extract ONLY snippets with:
        • Real email addresses
        • Real hiring context
        • Recruiter-level keywords
    """,

    tools=[search_tool],
    llm=agent_llm,
    verbose=True
)

# 3) LEAD VERIFIER
lead_verifier = Agent(
    role="Lead Verification & Data Cleaner",

    goal="""
    Validate recruiter emails with 100% strictness.
    Reject any email that belongs to a job seeker, student, or comment.
    Accept ONLY professional hiring contacts.
    """,

    backstory="""
    You must perform 3-level validation:

    LEVEL 1 — Email Pattern Check
        Accept only:
            • gmail/outlook/yahoo emails used by RECRUITERS
            • company emails containing:
                  hiring@, jobs@, careers@, hr@
        Reject if:
            • it looks personal with no hiring context
            • email belongs to a student (name+123, random digits)
            • snippet contains spam phrases

    LEVEL 2 — Recruiter Phrase Verification
        Snippet MUST contain at least one:
            ["HR", "Recruiter", "Talent Acquisition",
             "Hiring Manager", "HR Team", "Careers", "Jobs"]

    LEVEL 3 — Spam Filter
        Reject if snippet contains any:
            ["interested", "please sir", "my resume", "hire me",
             "student", "fresher", "looking for job", "need work"]

    Only if ALL 3 levels pass:
        Save using the CSV tool in format:
        "Company|Email|Role|Context|Source"
    """,

    tools=[csv_tool],
    llm=agent_llm,
    verbose=True
)

# 4) EMAIL WRITER
email_writer = Agent(
    role="Personalized Cold Email Writer",
    goal="Write short, punchy, role-specific cold emails based on verified leads.",
    
    backstory="""
    You tailor each email to the job role, recruiter type, and company domain.
    You map candidate strengths from resume.txt with job role requirements.
    """,
    
    tools=[file_tool],
    llm=agent_llm,
    verbose=True
)


In [12]:
# ====================================================
# 5. TASKS  (FULL UPGRADED VERSION)
# ====================================================

# TASK 1 — ROLE DISCOVERY
task_roles = Task(
    description="""
    1. Read resume.txt
    2. Identify candidate strengths + skills
    3. Infer top 3–5 suitable job roles
    4. Generate platform-specific search keywords + Boolean queries for each role
    """,
    expected_output="Role list + keyword clusters.",
    agent=role_discovery
)


# TASK 2 — MULTI-PLATFORM LEAD HUNTING
task_hunt = Task(
    description="""
    Use ONLY these recruiter-focused search templates (MUST USE):

1. site:linkedin.com/posts ("HR Manager" OR "Recruiter" OR "Talent Acquisition" OR "Hiring Manager") "<ROLE_KEYWORD>" "@gmail.com"
2. site:linkedin.com/in ("HR" OR "Recruiter" OR "Talent Acquisition") ("hiring" OR "opening") "<ROLE_KEYWORD>" "@gmail.com"
3. site:linkedin.com/company ("hiring" OR "we are hiring") "<ROLE_KEYWORD>" "@gmail.com"
4. site:naukri.com ("apply" OR "send resume") "<ROLE_KEYWORD>" "@gmail.com"
5. site:indeed.com "contact" "<ROLE_KEYWORD>" "@gmail.com"
6. site:foundit.in ("email" OR "contact") "<ROLE_KEYWORD>" "@gmail.com"
7. "<ROLE_KEYWORD>" "Hiring Manager" "email" "@gmail.com"
8. "<ROLE_KEYWORD>" "Talent Acquisition" "@gmail.com"
9. "<ROLE_KEYWORD>" "HR" "email" "@gmail.com"
10. "<ROLE_KEYWORD>" "Founder" "hiring" "@gmail.com"

Reject all results where:
• the snippet contains keywords from the spam list:
  ("interested", "please hire", "looking for job", "I want this job")

Accept only when:
• snippet contains a recruiter keyword
• email appears clearly as part of a hiring post

    """,
    expected_output="Raw snippets with emails.",
    agent=lead_hunter,
    context=[task_roles]
)


# TASK 3 — VERIFICATION AND CLEANING
task_verify = Task(
    description="""
    1. Parse all raw snippets.
    2. For each snippet:
         a. Extract clean email using regex
         b. Confirm hiring keywords are present
         c. Identify the associated job role
         d. Identify company name if visible
         e. Save using Lead Saver tool in format:
            "Company|Email|Role|Context|Source"
    3. Return all saved leads.
    """,
    expected_output="CSV updated with verified recruiter leads.",
    agent=lead_verifier,
    context=[task_hunt]
)


# TASK 4 — PERSONALIZED EMAIL WRITING
task_write = Task(
    description="""
    Generate EXACTLY 5 personalized cold emails for the verified leads.
    Emails must be:
      • short (80–120 words)
      • role-specific
      • recruiter-friendly
    Save the output in cold_emails.md.
    """,
    expected_output="cold_emails.md generated.",
    agent=email_writer,
    context=[task_verify],
    output_file="cold_emails.md"
)


In [13]:
# ====================================================
# 6. EXECUTION
# ====================================================

crew = Crew(
    agents=[lead_hunter, lead_verifier, email_writer],
    tasks=[task_hunt, task_verify, task_write],
    process=Process.sequential
)

print("🚀 Starting CSV Hunter Crew...")
result = crew.kickoff()

print(f"\n\n✅ DONE. Leads saved to '{csv_path}' and emails to 'cold_emails.md'")

🚀 Starting CSV Hunter Crew...


╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Recruiter Email Extraction Specialist                                                                   │
│                                                                                                                 │
│  Task:                                                                                                          │
│      Use ONLY these recruiter-focused search templates (MUST USE):                                              │
│                                                                                                                 │
│  1. site:linkedin.com/posts ("HR Manager" OR "Recruiter" OR "Talent Acquisition" OR "Hiring Manager")           │
│  "<ROLE_KEYWORD>" "@gmail.com"                                                                                  │
│  2. site:linkedin.com/in ("HR" OR "Recruiter" OR "Talent Acquisition") ("hiring" OR "opening")                  │
│  "<ROLE_KEYWORD>" "@gmail.com"                                                                                  │
│  3. site:linkedin.com/company ("hiring" OR "we are hiring") "<ROLE_KEYWORD>" "@gmail.com"                       │
│  4. site:naukri.com ("apply" OR "send resume") "<ROLE_KEYWORD>" "@gmail.com"                                    │
│  5. site:indeed.com "contact" "<ROLE_KEYWORD>" "@gmail.com"                                                     │
│  6. site:foundit.in ("email" OR "contact") "<ROLE_KEYWORD>" "@gmail.com"                                        │
│  7. "<ROLE_KEYWORD>" "Hiring Manager" "email" "@gmail.com"                                                      │
│  8. "<ROLE_KEYWORD>" "Talent Acquisition" "@gmail.com"                                                          │
│  9. "<ROLE_KEYWORD>" "HR" "email" "@gmail.com"                                                                  │
│  10. "<ROLE_KEYWORD>" "Founder" "hiring" "@gmail.com"                                                           │
│                                                                                                                 │
│  Reject all results where:                                                                                      │
│  • the snippet contains keywords from the spam list:                                                            │
│    ("interested", "please hire", "looking for job", "I want this job")                                          │
│                                                                                                                 │
│  Accept only when:                                                                                              │
│  • snippet contains a recruiter keyword                                                                         │
│  • email appears clearly as part of a hiring post                                                               │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Recruiter Email Extraction Specialist                                                                   │
│                                                                                                                 │
│  Thought: Thought:The user wants me to find recruiter emails for a specific role using a set of predefined      │
│  search templates and strict filtering rules.                                                                   │
│  The role keyword is missing from the prompt. I will use "Software Engineer" as a placeholder role to           │
│  demonstrate the process and execute the searches.                                                              │
│  I need to iterate through each of the 10 provided search templates, replace `<ROLE_KEYWORD>` with "Software    │
│  Engineer", perform the search using the `Search the internet with Serper` tool, and then carefully filter the  │
│  results based on the "STRICT FILTERING" and "REJECT" criteria.                                                 │
│  I will only accept snippets that contain recruiter indicators and an email, and reject any that contain spam   │
│  indicators.                                                                                                    │
│  Finally, I will return the raw, complete snippets that meet the criteria.                                      │
│  Let's start with template 1.                                                                                   │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  {                                                                                                              │
│    "search_query": "site:linkedin.com/posts (\"HR Manager\" OR \"Recruiter\" OR \"Talent Acquisition\" OR       │
│  \"Hiring Manager\") \"Software Engineer\" \"@gmail.com\""                                                      │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {'searchParameters': {'q': 'site:linkedin.com/posts ("HR Manager" OR "Recruiter" OR "Talent Acquisition" OR    │
│  "Hiring Manager") "Software Engineer" "@gmail.com"', 'type': 'search', 'num': 15, 'engine': 'google'},         │
│  'organic': [{'title': 'Beware of fake recruiters and resume scams via email', 'link':                          │
│  'https://www.linkedin.com/posts/patrick-j-omeara_scam-alert-be-alert-for-any-recruiter-activity-7362436010149  │
│  863424-asye', 'snippet': 'SCAM ALERT** Be alert for any recruiter that reaches out to you by email and claims  │
│  you are a good candidate for a specific high paying job ...', 'position': 1}, {'title': 'Be cautious of fake   │
│  recruiters on LinkedIn', 'link':                                                                               │
│  'https://www.linkedin.com/posts/thomasrmcdonald_beware-of-fake-recruiter-scams-on-linkedin-activity-736870460  │
│  5540577284-Bwoc', 'snippet': "... Talent Acquisition Partners' at a well-known recruiter. Hiring ...           │
│  gmail.com ✓ Go to the company website directly Find their careers page.", 'position': 2}, {'title': "Vanessa   │
│  Branca's Post", 'link':                                                                                        │
│  'https://www.linkedin.com/posts/vanessabranca_contract-complete-open-to-new-opportunities-activity-7311099732  │
│  624396290-Cc8I', 'snippet': '... Recruiter/Talent Acquisition Manager- My expertise lies in advanced ...       │
│  gmail.com Special shout out to my amazing managers who truly ...', 'position': 3}, {'title': 'Senior Talent    │
│  Acquisition Manager (Full- ...', 'link':                                                                       │
│  'https://www.linkedin.com/posts/dhanraj-shinemaven_senior-talent-acquisition-manager-full-time-activity-73697  │
│  62578991935488-uFEn', 'snippet': '... Recruiter & modern hiring tools • Proven experience in startup or ... I  │
│  am reachable @ 9620689897 / roopamchd@gmail.com.. I am ...', 'position': 4}, {'title': 'Candidate asked for    │
│  verification before accepting offer, a ...', 'link':                                                           │
│  'https://www.linkedin.com/posts/gabriela-de-luca_jobsearch-scamawareness-recruiting-activity-7368694228413083  │
│  648-kb-G', 'snippet': '... gmail.com or @outlook.com. 4️⃣ Requests for personal data ... Research the            │
│  Recruiter/Hiring Manager: Look up the indi...                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Personalized Cold Email Writer                                                                          │
│                                                                                                                 │
│  Thought: Action: Read a file's content                                                                         │
│                                                                                                                 │
│  Using Tool: Read a file's content                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  {                                                                                                              │
│    "file_path": "resume.txt"                                                                                    │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Hardik Sharma                                                                                                  │
│  HarShar017                                                                                                     │
│  Education                                                                                                      │
│  Experience                                                                                                     │
│  TechLead@MLSAC-BVP&TechExecutive@BVP-ISTE 2024â€“Present                                                       │
│  â€¢                                                                                                            │
│  Ledtechnicalprojects,hackathons,andeventswhilementoringjuniorsandmanagingdevelopmentinitiativesunderbothsocie  │
│  ties.                                                                                                          │
│  AI/MLBootcampâ€“5-DayProgram(Day4SessionLead) Nov2025                                                          │
│  â€¢ ConductedalivesessiononModelTraining:BasictoAdvanced,guiding100+studentsthroughend-to-endMLworkflows.      │
│  Projects                                                                                                       │
│  SkilledGuildâ€“CollaborativeSkill-BasedDevPlatform2 Django,TailwindCSS,JS                                      │
│  â€¢ Platformenablingdeveloperstojoinguildsandteamupforhackathonsorprojects.                                    │
│  â€¢ Dark-themedUIwithreversedlayoutlogic,forums,andskill-buildingquests.                                       │
│  TRAVERAâ€“AI-PoweredTravelAssistant2 HTML,CSS,React.js                                                         │
│  â€¢ Personalizedtravelplannerwithbudgettrackingandreal-timeupdates.                                            │
│  â€¢ IncludesAI-drivenhiddengemdiscoveryandmultilingualsupport.                                                 │
│  MusiKingâ€“MinimalistMusicPlayer2 HTML,CSS,JS                                                                  │
│  â€¢ Glassmorphic-themedmusicplayerwithsmoothcontrolsanddynamicplaylistupdates.                                 │
│  â€¢ Includesauthenticationsystemforpersonalizedplaylists.                                                      │
│  Achievements&Hackathons                                                                                        │
│  CodeJamTalks2025 1stPlace                                                                                      │
│  â€¢ WonfirstplaceinaDSA&CP-basedcompetitionamong200+participantsfroma25,000prizepool.                          │
│  Technotic2.0(CodeChefBVCOE) Top10                                                                              │
│  â€¢ Onlyfirst-yearteamtoreachfinalsamongseniorcompetitorsindebuggingandDSAchallenges.                          │
│  TechTitans2025 SpecialMention                                                                                  │
│  â€¢ Earnedrecognitionandprizesforinnovativeproblem-solvingandcreativesolutiondesign.                           │
│  HackWithIndia Top5,000Teams                                                                                    │
│  â€¢ RankedamongTop5,000of25,000teams;presentedTRAVERAduringthenationalpitchround.                              │
│  Certifications                                                                                                 │
│  SummerAnalyticsâ€™25â€“IITGuwahati                                                                             │
│  -6-weekintensiveprogramonDataScienceandMachineLearning.RankedinTop10%andcompletedareal-worldcapstoneproject.   │
│  CreatingAIAgentsUsingAgentforceâ€“GFGÃ—Salesforce    

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Personalized Cold Email Writer                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```                                                                                                            │
│  ### Email 1: Wiselabs (Freelance Software Engineer)                                                            │
│                                                                                                                 │
│  **To:** wiselabs.management@gmail.com                                                                          │
│  **Subject:** Freelance Software Engineer - Hardik Sharma (Python/Django)                                       │
│                                                                                                                 │
│  Dear Wiselabs Hiring Team,                                                                                     │
│                                                                                                                 │
│  I'm Hardik Sharma, a Software Engineer with strong skills in Python and Django, and I noticed your interest    │
│  in freelance software engineers. My experience includes leading technical projects and developing platforms    │
│  like "SkilledGuild," a collaborative dev platform built with Django and TailwindCSS.                           │
│                                                                                                                 │
│  I'm adept at creating robust, scalable web applications and am passionate about delivering high-quality code.  │
│  My background in AI/ML also brings an additional layer of problem-solving capabilities.                        │
│                                                                                                                 │
│  I'd be keen to discuss how my skills could contribute to your freelance projects. Please find my portfolio on  │
│  GitHub (HarShar017) for a closer look at my work.                                                              │
│                                                                                                                 │
│  Best regards,                                                                                                  │
│                                                                                                                 │
│  Hardik Sharma                                                                                                  │
│  hardiksharma.hs17@gmail.com                                                                                    │
│  LinkedIn: linkedin.com/in/hardik-sharma-hs17/                                                                  │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### Email 2: PT Daya Rekadigital Indonesia (Software Engineer)                                                 │
│                                                                                                                 │
│  **To:** rekrutmen.dri@gmail.com                                                                                │
│  **Subject:** Software Engineer Application - Hardik Sh



✅ DONE. Leads saved to 'leads.csv' and emails to 'cold_emails.md'
